In [1]:
# Importing libraries

# These are the normal libraries
import pandas as pd
import numpy as np

# This is just so that we don't get annoying warnings
import warnings
warnings.filterwarnings('ignore')

# This is the most common viz library in python
import matplotlib.pyplot as plt
%matplotlib inline

# This one is the above on steroids
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# These Libs are for stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [2]:
# Clean and fill the NA value and order per month

url = 'https://docs.google.com/spreadsheets/d/1a9g5MfgAJ17HiYYKsjBi2yaO_HMGLIAX/edit?usp=sharing&ouid=108054823958769193495&rtpof=true&sd=true'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
mark_cust_A = pd.read_excel(path)

# fonction processing all the steps to make the data clean for our file
def Data_quarter_Y(mark_cust_A):
    # creation of a list to store columns name after lower and removing spaces
    cols_st = []
    # 
    for i in range(len(mark_cust_A.columns)):
        cols_st.append(mark_cust_A.columns[i].lower().replace(" ","_"))
    mark_cust_A.columns = cols_st
    mark_cust_A = mark_cust_A.rename(columns = {"employmentstatus":"employment_status"})
    mark_cust_A = mark_cust_A.drop({"unnamed:_0","state" },axis=1)
    mark_cust_A = mark_cust_A.fillna({'response':'No','number_of_open_complaints': 0,'vehicle_type':'Other vehicule','months_since_last_claim':mark_cust_A['months_since_last_claim'].mean(),'vehicle_size':'Medsize'})
    mark_cust_A['date'] = pd.to_datetime(mark_cust_A['effective_to_date'])
    mark_cust_A['month'] = mark_cust_A['date'].dt.to_period('M')
    return mark_cust_A
    
mark_cust_A = Data_quarter_Y(mark_cust_A)

mark_cust_A


,customer,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,location_code,...,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,date,month
0,DK49336,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,Suburban,...,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Other vehicule,2011-02-18,2011-02
1,KX64629,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,Suburban,...,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Other vehicule,2011-01-18,2011-01
2,LZ68649,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,Suburban,...,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2011-02-10,2011-02
3,XL78013,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,Suburban,...,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01-11,2011-01
4,QA50777,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,Suburban,...,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Other vehicule,2011-01-17,2011-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,Suburban,...,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,2011-01-19,2011-01
10906,KX53892,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,Urban,...,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,2011-01-06,2011-01
10907,TL39050,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,Rural,...,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,Other vehicule,2011-02-06,2011-02
10908,WA60547,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,Urban,...,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2011-02-13,2011-02


In [3]:
mark_cust_A.isna().sum()
mark_cust_A['vehicle_class'].unique()
mark_cust_A['vehicle_class'] = mark_cust_A['vehicle_class'].fillna('Car')
mark_cust_A

,customer,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,location_code,...,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,date,month
0,DK49336,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,Suburban,...,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Other vehicule,2011-02-18,2011-02
1,KX64629,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,Suburban,...,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Other vehicule,2011-01-18,2011-01
2,LZ68649,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,Suburban,...,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2011-02-10,2011-02
3,XL78013,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,Suburban,...,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01-11,2011-01
4,QA50777,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,Suburban,...,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Other vehicule,2011-01-17,2011-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,Suburban,...,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,2011-01-19,2011-01
10906,KX53892,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,Urban,...,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,2011-01-06,2011-01
10907,TL39050,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,Rural,...,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,Other vehicule,2011-02-06,2011-02
10908,WA60547,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,Urban,...,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2011-02-13,2011-02


In [4]:
Numerical = mark_cust_A.select_dtypes(include='number').reset_index()


Categorical = mark_cust_A.select_dtypes(include=object).reset_index()

Numerical

,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,4809.216960,48029,61,7.000000,52,0.0,9,292.800000
1,1,2228.525238,0,64,3.000000,26,0.0,1,744.924331
2,2,14947.917300,22139,100,34.000000,31,0.0,2,480.000000
3,3,22332.439460,49078,97,10.000000,3,0.0,2,484.013411
4,4,9025.067525,23675,117,15.149071,31,0.0,7,707.925645
...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.0,7,1214.400000
10906,10906,5259.444853,61146,65,7.000000,68,0.0,6,273.018929
10907,10907,23893.304100,39837,201,11.000000,63,0.0,2,381.306996
10908,10908,11971.977650,64195,158,0.000000,27,4.0,6,618.288849


In [5]:


Categorical = Categorical.drop(['customer'], axis = 1)
# chane value for object to 0 and 1
New_Categorical = pd.get_dummies(Categorical)


New_df = pd.concat([New_Categorical, Numerical], axis = 1).reset_index()

New_df = New_df.drop(['level_0','index'], axis = 1)

New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,1,0,0,0,1,0,0,0,...,0,1,4809.216960,48029,61,7.000000,52,0.0,9,292.800000
1,1,0,1,0,0,0,1,0,0,0,...,0,1,2228.525238,0,64,3.000000,26,0.0,1,744.924331
2,1,0,1,0,0,1,0,0,0,0,...,1,0,14947.917300,22139,100,34.000000,31,0.0,2,480.000000
3,0,1,0,1,0,0,1,0,0,0,...,1,0,22332.439460,49078,97,10.000000,3,0.0,2,484.013411
4,1,0,0,0,1,1,0,0,0,0,...,0,1,9025.067525,23675,117,15.149071,31,0.0,7,707.925645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1,0,0,0,1,1,0,0,0,0,...,1,0,15563.369440,0,253,15.149071,40,0.0,7,1214.400000
10906,1,0,1,0,0,0,1,0,0,0,...,1,0,5259.444853,61146,65,7.000000,68,0.0,6,273.018929
10907,1,0,0,1,0,1,0,0,0,0,...,0,1,23893.304100,39837,201,11.000000,63,0.0,2,381.306996
10908,1,0,0,0,1,0,1,0,0,0,...,1,0,11971.977650,64195,158,0.000000,27,4.0,6,618.288849


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test_poly)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.511219037401142e+21
Mean Squared Error (MSE) = 2.0276329572238874e+26
Root Mean Squared Error (RMSE) = 14239497734203.58
Mean Absolute Error (MAE) = 487949827952.96


In [7]:
# I will round numerical numbers to get better result 

Numerical
Numerical['customer_lifetime_value'] = Numerical['customer_lifetime_value'].round(1)

Numerical['total_claim_amount'] = Numerical['total_claim_amount'].round(1)

Numerical['months_since_last_claim'] = Numerical['months_since_last_claim'].round(1)






New_df = pd.concat([New_Categorical, Numerical], axis = 1)



New_df






,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,...,vehicle_type_Other vehicule,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,1,0,1,0,0,0,1,0,0,...,1,0,4809.2,48029,61,7.0,52,0.0,9,292.8
1,1,1,0,1,0,0,0,1,0,0,...,1,1,2228.5,0,64,3.0,26,0.0,1,744.9
2,2,1,0,1,0,0,1,0,0,0,...,0,2,14947.9,22139,100,34.0,31,0.0,2,480.0
3,3,0,1,0,1,0,0,1,0,0,...,0,3,22332.4,49078,97,10.0,3,0.0,2,484.0
4,4,1,0,0,0,1,1,0,0,0,...,1,4,9025.1,23675,117,15.1,31,0.0,7,707.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,1,0,0,0,1,1,0,0,0,...,0,10905,15563.4,0,253,15.1,40,0.0,7,1214.4
10906,10906,1,0,1,0,0,0,1,0,0,...,0,10906,5259.4,61146,65,7.0,68,0.0,6,273.0
10907,10907,1,0,0,1,0,1,0,0,0,...,1,10907,23893.3,39837,201,11.0,63,0.0,2,381.3
10908,10908,1,0,0,0,1,0,1,0,0,...,0,10908,11972.0,64195,158,0.0,27,4.0,6,618.3


In [8]:
# give much better result with round numerique number

# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test_poly)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))






R-squared (R2) = -1.5792958419868697e+22
Mean Squared Error (MSE) = 1.2751765292519654e+27
Root Mean Squared Error (RMSE) = 35709613961116.48
Mean Absolute Error (MAE) = 1308746679954.15


In [9]:
# do correlation matrcie to see the more realted value 

correlations_matrix = New_df.corr()
correlations_matrix = pd.DataFrame(correlations_matrix['total_claim_amount'])
correlations_matrix['total_claim_amount'] = np.absolute(correlations_matrix['total_claim_amount'])
correlations_matrix = correlations_matrix.sort_values(by = ['total_claim_amount'], ascending=False)
correlations_matrix.iloc[0:30,:]


,total_claim_amount
total_claim_amount,1.000000
monthly_premium_auto,0.631798
location_code_Suburban,0.576227
location_code_Rural,0.544382
income,0.354143
employment_status_Employed,0.336130
employment_status_Unemployed,0.332942
vehicle_class_Luxury Car,0.317039
vehicle_class_Luxury SUV,0.308382
vehicle_class_Four-Door Car,0.270956


In [10]:
# creating new column , to try to get the formula for total claim amount,
# error is worst with these column so I delete the code
# New_df['cust_pay'] = New_df['monthly_premium_auto'] * New_df['months_since_policy_inception']

New_df

,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,...,vehicle_type_Other vehicule,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,1,0,1,0,0,0,1,0,0,...,1,0,4809.2,48029,61,7.0,52,0.0,9,292.8
1,1,1,0,1,0,0,0,1,0,0,...,1,1,2228.5,0,64,3.0,26,0.0,1,744.9
2,2,1,0,1,0,0,1,0,0,0,...,0,2,14947.9,22139,100,34.0,31,0.0,2,480.0
3,3,0,1,0,1,0,0,1,0,0,...,0,3,22332.4,49078,97,10.0,3,0.0,2,484.0
4,4,1,0,0,0,1,1,0,0,0,...,1,4,9025.1,23675,117,15.1,31,0.0,7,707.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,1,0,0,0,1,1,0,0,0,...,0,10905,15563.4,0,253,15.1,40,0.0,7,1214.4
10906,10906,1,0,1,0,0,0,1,0,0,...,0,10906,5259.4,61146,65,7.0,68,0.0,6,273.0
10907,10907,1,0,0,1,0,1,0,0,0,...,1,10907,23893.3,39837,201,11.0,63,0.0,2,381.3
10908,10908,1,0,0,0,1,0,1,0,0,...,0,10908,11972.0,64195,158,0.0,27,4.0,6,618.3


In [11]:
# same here

#New_df['customer_value'] = New_df['cust_pay'] - New_df['total_claim_amount'] 

#New_df['customer_value']

New_df

,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,...,vehicle_type_Other vehicule,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,1,0,1,0,0,0,1,0,0,...,1,0,4809.2,48029,61,7.0,52,0.0,9,292.8
1,1,1,0,1,0,0,0,1,0,0,...,1,1,2228.5,0,64,3.0,26,0.0,1,744.9
2,2,1,0,1,0,0,1,0,0,0,...,0,2,14947.9,22139,100,34.0,31,0.0,2,480.0
3,3,0,1,0,1,0,0,1,0,0,...,0,3,22332.4,49078,97,10.0,3,0.0,2,484.0
4,4,1,0,0,0,1,1,0,0,0,...,1,4,9025.1,23675,117,15.1,31,0.0,7,707.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,1,0,0,0,1,1,0,0,0,...,0,10905,15563.4,0,253,15.1,40,0.0,7,1214.4
10906,10906,1,0,1,0,0,0,1,0,0,...,0,10906,5259.4,61146,65,7.0,68,0.0,6,273.0
10907,10907,1,0,0,1,0,1,0,0,0,...,1,10907,23893.3,39837,201,11.0,63,0.0,2,381.3
10908,10908,1,0,0,0,1,0,1,0,0,...,0,10908,11972.0,64195,158,0.0,27,4.0,6,618.3


In [12]:
Categorical

,index,response,coverage,education,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,vehicle_type
0,0,No,Basic,College,Employed,M,Suburban,Married,Corporate Auto,Corporate L3,Offer3,Agent,Four-Door Car,Medsize,Other vehicule
1,1,No,Basic,College,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer4,Call Center,Four-Door Car,Medsize,Other vehicule
2,2,No,Basic,Bachelor,Employed,M,Suburban,Single,Personal Auto,Personal L3,Offer3,Call Center,SUV,Medsize,A
3,3,Yes,Extended,College,Employed,M,Suburban,Single,Corporate Auto,Corporate L3,Offer2,Branch,Four-Door Car,Medsize,A
4,4,No,Premium,Bachelor,Medical Leave,F,Suburban,Married,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Medsize,Other vehicule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,No,Premium,Bachelor,Unemployed,F,Suburban,Married,Personal Auto,Personal L1,Offer3,Web,Luxury Car,Medsize,A
10906,10906,No,Basic,College,Employed,F,Urban,Married,Personal Auto,Personal L3,Offer2,Branch,Four-Door Car,Medsize,A
10907,10907,No,Extended,Bachelor,Employed,F,Rural,Married,Corporate Auto,Corporate L3,Offer1,Web,Luxury SUV,Medsize,Other vehicule
10908,10908,No,Premium,College,Employed,F,Urban,Divorced,Personal Auto,Personal L1,Offer1,Branch,SUV,Medsize,A


In [13]:

correlations_matrix = New_df.corr()
correlations_matrix = pd.DataFrame(correlations_matrix['total_claim_amount'])
correlations_matrix['total_claim_amount'] = np.absolute(correlations_matrix['total_claim_amount'])
correlations_matrix = correlations_matrix.sort_values(by = ['total_claim_amount'], ascending=False)
correlations_matrix.iloc[0:70,:]


,total_claim_amount
total_claim_amount,1.000000
monthly_premium_auto,0.631798
location_code_Suburban,0.576227
location_code_Rural,0.544382
income,0.354143
...,...
policy_Personal L3,0.000869
index,0.000593
index,0.000593
sales_channel_Web,0.000466


In [14]:
# filtering by groupe of location

condition = New_df['location_code_Rural'] == 1

Df_code_rural = New_df[condition]

Df_code_rural.reset_index()



,level_0,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,...,vehicle_type_Other vehicule,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,12,12,1,0,0,1,0,0,0,0,...,1,12,19955.3,69096,99,24.0,32,0.0,2,64.7
1,19,19,0,1,0,1,0,0,1,0,...,0,19,3746.8,41479,94,14.0,38,1.0,1,19.6
2,23,23,1,0,1,0,0,1,0,0,...,1,23,5107.1,70174,64,15.1,80,0.0,7,128.9
3,25,25,1,0,0,1,0,0,1,0,...,0,25,9521.2,27769,121,24.0,78,0.0,6,123.2
4,26,26,1,0,1,0,0,0,1,0,...,0,26,8869.6,81350,110,1.0,28,0.0,9,228.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,10890,10890,1,0,1,0,0,1,0,0,...,1,10890,4660.4,89834,116,24.0,11,0.0,1,43.2
2112,10893,10893,1,0,1,0,0,0,1,0,...,1,10893,4818.0,38157,61,2.0,39,0.0,4,81.5
2113,10897,10897,0,1,1,0,0,0,0,0,...,0,10897,9075.8,37722,116,8.0,23,0.0,8,158.1
2114,10898,10898,1,0,1,0,0,0,1,0,...,1,10898,5807.7,62447,74,9.0,2,0.0,7,134.0


In [15]:
# error is worst with this data
# Df_code_rural.drop(['level_0','index'],axis = 1)
# Split the data into features (X) and the target variable (Y)
Y = Df_code_rural['total_claim_amount']
X = Df_code_rural.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test_poly)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -7.375836191740724e+24
Mean Squared Error (MSE) = 4.591250655190185e+28
Root Mean Squared Error (RMSE) = 214272038660908.47
Mean Absolute Error (MAE) = 43452253182249.21


In [16]:
Df_code_rural

,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,...,vehicle_type_Other vehicule,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
12,12,1,0,0,1,0,0,0,0,1,...,1,12,19955.3,69096,99,24.0,32,0.0,2,64.7
19,19,0,1,0,1,0,0,1,0,0,...,0,19,3746.8,41479,94,14.0,38,1.0,1,19.6
23,23,1,0,1,0,0,1,0,0,0,...,1,23,5107.1,70174,64,15.1,80,0.0,7,128.9
25,25,1,0,0,1,0,0,1,0,0,...,0,25,9521.2,27769,121,24.0,78,0.0,6,123.2
26,26,1,0,1,0,0,0,1,0,0,...,0,26,8869.6,81350,110,1.0,28,0.0,9,228.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10890,10890,1,0,1,0,0,1,0,0,0,...,1,10890,4660.4,89834,116,24.0,11,0.0,1,43.2
10893,10893,1,0,1,0,0,0,1,0,0,...,1,10893,4818.0,38157,61,2.0,39,0.0,4,81.5
10897,10897,0,1,1,0,0,0,0,0,0,...,0,10897,9075.8,37722,116,8.0,23,0.0,8,158.1
10898,10898,1,0,1,0,0,0,1,0,0,...,1,10898,5807.7,62447,74,9.0,2,0.0,7,134.0


In [17]:
# not really woprking

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score

import statsmodels.api as sm
from statsmodels.formula.api import ols

Y = Df_code_rural['total_claim_amount']
X = Df_code_rural.drop(['total_claim_amount'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# the mod el = Y = inter + coef. * X

X = sm.add_constant(X)

# ordinary least square

model = sm.OLS(y_train,X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     12.94
Date:                Mon, 06 Nov 2023   Prob (F-statistic):           1.25e-77
Time:                        20:42:01   Log-Likelihood:                -9488.1
No. Observations:                1692   AIC:                         1.907e+04
Df Residuals:                    1647   BIC:                         1.931e+04
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
index                              -0.0002      0.000     -0.652      0.514      -0.001       0.000
response_No                        -2.9250      3.970     -0.737      0.461     -10.712       4.862
response_Yes                       -4.9169      4.507     -1.091      0.276     -13.758       3.924
coverage_Basic                     14.8518      3.679      4.037      0.000       7.636      22.068
coverage_Extended                  -3.7085      3.237     -1.146      0.252     -10.057       2.640
coverage_Premium                  -18.9852      7.961     -2.385      0.017     -34.600      -3.370
education_Bachelor                 -2.7404      3.225     -0.850      0.396      -9.067       3.586
education_College                  -5.7975      3.233     -1.793      0.073     -12.138       0.543
education_Doctor                    4.4185      6.031      0.733      0.464      -7.411      16.248
education_High School or Below     -6.6576      3.701     -1.799      0.072     -13.917       0.602
education_Master                    2.9352      4.283      0.685      0.493      -5.465      11.336
employment_status_Disabled         -9.6428      9.112     -1.058      0.290     -27.515       8.230
employment_status_Employed          9.2813      5.685      1.633      0.103      -1.869      20.431
employment_status_Medical Leave    -3.7081     10.463     -0.354      0.723     -24.231      16.814
employment_status_Retired       -1.302e-11   5.01e-12     -2.599      0.009   -2.28e-11   -3.19e-12
employment_status_Unemployed       -3.7721      6.918     -0.545      0.586     -17.341       9.797
gender_F                           -7.8569      3.322     -2.365      0.018     -14.373      -1.340
gender_M                            0.0151      3.388      0.004      0.996      -6.630       6.660
location_code_Rural                -7.8418      5.798     -1.353      0.176     -19.213       3.530
location_code_Suburban          -6.975e-15   8.42e-15     -0.829      0.407   -2.35e-14    9.53e-15
location_code_Urban              3.565e-15   6.15e-15      0.580      0.562    -8.5e-15    1.56e-14
marital_status_Divorced            -2.5857      3.754     -0.689      0.491      -9.949       4.778
marital_status_Married             -3.4782      2.975     -1.169      0.243      -9.313       2.357
marital_status_Single              -1.7780      3.850     -0.462      0.644      -9.328       5.773
policy_type_Corporate Auto         -5.8460      3.443     -1.698      0.090     -12.600       0.908
policy_type_Personal Auto          -4.5441      3.059     -1.485      0.138     -10.544       1.456
policy_type_Special Auto            2.5483      4.776      0.534      0.594      -6.820      11.916
policy_Corporate L1                 6.5518      6.393      1.025      0.306      -5

In [18]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Calculating errors
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_predictions = lm.predict(X_test)

r2 = r2_score(y_test, y_predictions)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_predictions)

print("R2 value is =", round(r2, 4))
print("The mean squared error of the model is =", round(mse, 4))
print("The root mean squared error of the model is =", round(rmse, 4))
print("The mean absolute error of the model is =", round(mae, 4))

R2 value is = 0.2166
The mean squared error of the model is = 4876.1849
The root mean squared error of the model is = 69.8297
The mean absolute error of the model is = 57.0261


In [19]:
# filter by total claim amount

New_df = New_df.sort_values(by='total_claim_amount', ascending = False)
New_df = New_df.reset_index()





In [20]:
New_df = New_df.drop(['level_0','index'],axis = 1)

In [21]:
New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,0,1,0,1,0,0,0,0,...,1,0,8581.3,27689,239,0.0,72,2.0,1,2893.2
1,1,0,0,0,1,0,0,0,1,0,...,1,0,9295.4,0,261,33.0,44,0.0,1,2759.8
2,1,0,0,1,0,0,0,0,1,0,...,1,0,43217.2,51504,220,30.0,99,0.0,2,2552.3
3,1,0,0,1,0,0,0,0,1,0,...,0,1,43217.2,51504,220,30.0,99,0.0,2,2552.3
4,1,0,0,0,1,0,0,0,1,0,...,0,1,32104.7,89546,266,4.0,18,1.0,2,2452.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1,0,1,0,0,1,0,0,0,0,...,0,1,4538.8,82297,116,15.1,62,0.0,1,0.4
10906,1,0,0,1,0,0,1,0,0,0,...,0,1,6186.5,88375,77,23.0,11,0.0,3,0.4
10907,1,0,1,0,0,1,0,0,0,0,...,0,1,4538.8,82297,116,5.0,62,0.0,1,0.4
10908,1,0,0,1,0,0,1,0,0,0,...,1,0,6186.5,88375,77,23.0,11,0.0,3,0.4


In [22]:
# Selecting 2000 biggest claim amount rows 

New_df = New_df.iloc[0:2000,:]



In [23]:

New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,0,1,0,1,0,0,0,0,...,1,0,8581.3,27689,239,0.0,72,2.0,1,2893.2
1,1,0,0,0,1,0,0,0,1,0,...,1,0,9295.4,0,261,33.0,44,0.0,1,2759.8
2,1,0,0,1,0,0,0,0,1,0,...,1,0,43217.2,51504,220,30.0,99,0.0,2,2552.3
3,1,0,0,1,0,0,0,0,1,0,...,0,1,43217.2,51504,220,30.0,99,0.0,2,2552.3
4,1,0,0,0,1,0,0,0,1,0,...,0,1,32104.7,89546,266,4.0,18,1.0,2,2452.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,0,0,1,0,1,0,0,0,...,1,0,11972.0,64195,158,0.0,27,4.0,6,618.3
1996,1,0,0,1,0,1,0,0,0,0,...,0,1,10393.0,93495,129,8.0,0,0.0,9,618.2
1997,1,0,1,0,0,0,0,0,1,0,...,0,1,7322.9,80864,182,5.0,30,0.0,1,618.2
1998,1,0,1,0,0,1,0,0,0,0,...,0,1,8845.2,45600,113,6.0,96,0.0,3,617.9


In [24]:
# lets try the model now 

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score

import statsmodels.api as sm
from statsmodels.formula.api import ols

Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# the mod el = Y = inter + coef. * X

X = sm.add_constant(X)

# ordinary least square

model = sm.OLS(y_train,X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     42.35
Date:                Mon, 06 Nov 2023   Prob (F-statistic):          1.91e-234
Time:                        20:42:01   Log-Likelihood:                -10789.
No. Observations:                1600   AIC:                         2.167e+04
Df Residuals:                    1554   BIC:                         2.192e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
response_No                        24.2419     11.189      2.167      0.030       2.294      46.190
response_Yes                      -22.1094     12.417     -1.781      0.075     -46.465       2.246
coverage_Basic                     53.6369      9.834      5.454      0.000      34.348      72.926
coverage_Extended                 -24.2905      8.230     -2.951      0.003     -40.435      -8.147
coverage_Premium                  -27.2139     17.238     -1.579      0.115     -61.025       6.597
education_Bachelor                 24.8476     12.653      1.964      0.050       0.030      49.665
education_College                  -5.0047     13.039     -0.384      0.701     -30.580      20.571
education_Doctor                  -42.6071     33.640     -1.267      0.206    -108.592      23.378
education_High School or Below     38.9024     12.435      3.128      0.002      14.511      63.294
education_Master                  -14.0057     21.510     -0.651      0.515     -56.197      28.186
employment_status_Disabled          3.4523     24.242      0.142      0.887     -44.098      51.003
employment_status_Employed        -36.4783     17.580     -2.075      0.038     -70.962      -1.995
employment_status_Medical Leave    61.5162     21.224      2.898      0.004      19.885     103.147
employment_status_Retired         -63.0547     29.710     -2.122      0.034    -121.331      -4.779
employment_status_Unemployed       36.6969     15.964      2.299      0.022       5.384      68.010
gender_F                          -16.8734      8.841     -1.909      0.056     -34.214       0.468
gender_M                           19.0058      8.680      2.190      0.029       1.980      36.032
location_code_Rural              4.442e-14   2.04e-14      2.181      0.029    4.47e-15    8.44e-14
location_code_Suburban            152.8495     16.087      9.501      0.000     121.295     184.404
location_code_Urban              -150.7170     20.322     -7.417      0.000    -190.578    -110.856
marital_status_Divorced           -20.9338     13.458     -1.555      0.120     -47.332       5.465
marital_status_Married            -44.9323      9.695     -4.635      0.000     -63.948     -25.916
marital_status_Single              67.9986     10.850      6.267      0.000      46.717      89.280
policy_type_Corporate Auto         -5.2386      9.940     -0.527      0.598     -24.735      14.258
policy_type_Personal Auto          -2.3458      8.652     -0.271      0.786     -19.316      14.625
policy_type_Special Auto            9.7169     14.639      0.664      0.507     -18.997      38.431
policy_Corporate L1                 0.3358     20.037      0.017      0.987     -38.967      39.638
policy_Corporate L2                -4.3023     17.627     -0.244      0.807     -38

In [25]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Calculating errors
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_predictions = lm.predict(X_test)

r2 = r2_score(y_test, y_predictions)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_predictions)

print("R2 value is =", round(r2, 4))
print("The mean squared error of the model is =", round(mse, 4))
print("The root mean squared error of the model is =", round(rmse, 4))
print("The mean absolute error of the model is =", round(mae, 4))

R2 value is = 0.4997
The mean squared error of the model is = 37891.4472
The root mean squared error of the model is = 194.6573
The mean absolute error of the model is = 152.6274


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test_poly)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.1959145094386618e+24
Mean Squared Error (MSE) = 1.663156914500729e+29
Root Mean Squared Error (RMSE) = 407818208826031.8
Mean Absolute Error (MAE) = 74999805828641.48


In [32]:
# Stanrdarsin data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
New_df.columns

Index(['response_No', 'response_Yes', 'coverage_Basic', 'coverage_Extended',
       'coverage_Premium', 'education_Bachelor', 'education_College',
       'education_Doctor', 'education_High School or Below',
       'education_Master', 'employment_status_Disabled',
       'employment_status_Employed', 'employment_status_Medical Leave',
       'employment_status_Retired', 'employment_status_Unemployed', 'gender_F',
       'gender_M', 'location_code_Rural', 'location_code_Suburban',
       'location_code_Urban', 'marital_status_Divorced',
       'marital_status_Married', 'marital_status_Single',
       'policy_type_Corporate Auto', 'policy_type_Personal Auto',
       'policy_type_Special Auto', 'policy_Corporate L1',
       'policy_Corporate L2', 'policy_Corporate L3', 'policy_Personal L1',
       'policy_Personal L2', 'policy_Personal L3', 'policy_Special L1',
       'policy_Special L2', 'policy_Special L3', 'renew_offer_type_Offer1',
       'renew_offer_type_Offer2', 'renew_offer_type_O

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test_poly)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.1959145094386618e+24
Mean Squared Error (MSE) = 1.663156914500729e+29
Root Mean Squared Error (RMSE) = 407818208826031.8
Mean Absolute Error (MAE) = 74999805828641.48


In [39]:

Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# the mod el = Y = inter + coef. * X

X = sm.add_constant(X)

# ordinary least square

model = sm.OLS(y_train,X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     42.35
Date:                Mon, 06 Nov 2023   Prob (F-statistic):          1.91e-234
Time:                        20:48:43   Log-Likelihood:                -10789.
No. Observations:                1600   AIC:                         2.167e+04
Df Residuals:                    1554   BIC:                         2.192e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
response_No                        24.2419     11.189      2.167      0.030       2.294      46.190
response_Yes                      -22.1094     12.417     -1.781      0.075     -46.465       2.246
coverage_Basic                     53.6369      9.834      5.454      0.000      34.348      72.926
coverage_Extended                 -24.2905      8.230     -2.951      0.003     -40.435      -8.147
coverage_Premium                  -27.2139     17.238     -1.579      0.115     -61.025       6.597
education_Bachelor                 24.8476     12.653      1.964      0.050       0.030      49.665
education_College                  -5.0047     13.039     -0.384      0.701     -30.580      20.571
education_Doctor                  -42.6071     33.640     -1.267      0.206    -108.592      23.378
education_High School or Below     38.9024     12.435      3.128      0.002      14.511      63.294
education_Master                  -14.0057     21.510     -0.651      0.515     -56.197      28.186
employment_status_Disabled          3.4523     24.242      0.142      0.887     -44.098      51.003
employment_status_Employed        -36.4783     17.580     -2.075      0.038     -70.962      -1.995
employment_status_Medical Leave    61.5162     21.224      2.898      0.004      19.885     103.147
employment_status_Retired         -63.0547     29.710     -2.122      0.034    -121.331      -4.779
employment_status_Unemployed       36.6969     15.964      2.299      0.022       5.384      68.010
gender_F                          -16.8734      8.841     -1.909      0.056     -34.214       0.468
gender_M                           19.0058      8.680      2.190      0.029       1.980      36.032
location_code_Rural              4.442e-14   2.04e-14      2.181      0.029    4.47e-15    8.44e-14
location_code_Suburban            152.8495     16.087      9.501      0.000     121.295     184.404
location_code_Urban              -150.7170     20.322     -7.417      0.000    -190.578    -110.856
marital_status_Divorced           -20.9338     13.458     -1.555      0.120     -47.332       5.465
marital_status_Married            -44.9323      9.695     -4.635      0.000     -63.948     -25.916
marital_status_Single              67.9986     10.850      6.267      0.000      46.717      89.280
policy_type_Corporate Auto         -5.2386      9.940     -0.527      0.598     -24.735      14.258
policy_type_Personal Auto          -2.3458      8.652     -0.271      0.786     -19.316      14.625
policy_type_Special Auto            9.7169     14.639      0.664      0.507     -18.997      38.431
policy_Corporate L1                 0.3358     20.037      0.017      0.987     -38.967      39.638
policy_Corporate L2                -4.3023     17.627     -0.244      0.807     -38

In [50]:
# Evaluate the model with standardisation and with data ordered by 2000 ordered total_claim_amount
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.1959145094386618e+24
Mean Squared Error (MSE) = 1.663156914500729e+29
Root Mean Squared Error (RMSE) = 407818208826031.8
Mean Absolute Error (MAE) = 74999805828641.48


In [53]:
New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,0,1,0,1,0,0,0,0,...,1,0,8581.3,27689,239,0.0,72,2.0,1,2893.2
1,1,0,0,0,1,0,0,0,1,0,...,1,0,9295.4,0,261,33.0,44,0.0,1,2759.8
2,1,0,0,1,0,0,0,0,1,0,...,1,0,43217.2,51504,220,30.0,99,0.0,2,2552.3
3,1,0,0,1,0,0,0,0,1,0,...,0,1,43217.2,51504,220,30.0,99,0.0,2,2552.3
4,1,0,0,0,1,0,0,0,1,0,...,0,1,32104.7,89546,266,4.0,18,1.0,2,2452.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,0,0,1,0,1,0,0,0,...,1,0,11972.0,64195,158,0.0,27,4.0,6,618.3
1996,1,0,0,1,0,1,0,0,0,0,...,0,1,10393.0,93495,129,8.0,0,0.0,9,618.2
1997,1,0,1,0,0,0,0,0,1,0,...,0,1,7322.9,80864,182,5.0,30,0.0,1,618.2
1998,1,0,1,0,0,1,0,0,0,0,...,0,1,8845.2,45600,113,6.0,96,0.0,3,617.9


In [55]:


# Calculating errors
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_predictions = lm.predict(X_test)

r2 = r2_score(y_test, y_predictions)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_predictions)

print("R2 value is =", round(r2, 4))
print("The mean squared error of the model is =", round(mse, 4))
print("The root mean squared error of the model is =", round(rmse, 4))
print("The mean absolute error of the model is =", round(mae, 4))

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     42.35
Date:                Mon, 06 Nov 2023   Prob (F-statistic):          1.91e-234
Time:                        21:06:54   Log-Likelihood:                -10789.
No. Observations:                1600   AIC:                         2.167e+04
Df Residuals:                    1554   BIC:                         2.192e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
response_No                        24.2419     11.189      2.167      0.030       2.294      46.190
response_Yes                      -22.1094     12.417     -1.781      0.075     -46.465       2.246
coverage_Basic                     53.6369      9.834      5.454      0.000      34.348      72.926
coverage_Extended                 -24.2905      8.230     -2.951      0.003     -40.435      -8.147
coverage_Premium                  -27.2139     17.238     -1.579      0.115     -61.025       6.597
education_Bachelor                 24.8476     12.653      1.964      0.050       0.030      49.665
education_College                  -5.0047     13.039     -0.384      0.701     -30.580      20.571
education_Doctor                  -42.6071     33.640     -1.267      0.206    -108.592      23.378
education_High School or Below     38.9024     12.435      3.128      0.002      14.511      63.294
education_Master                  -14.0057     21.510     -0.651      0.515     -56.197      28.186
employment_status_Disabled          3.4523     24.242      0.142      0.887     -44.098      51.003
employment_status_Employed        -36.4783     17.580     -2.075      0.038     -70.962      -1.995
employment_status_Medical Leave    61.5162     21.224      2.898      0.004      19.885     103.147
employment_status_Retired         -63.0547     29.710     -2.122      0.034    -121.331      -4.779
employment_status_Unemployed       36.6969     15.964      2.299      0.022       5.384      68.010
gender_F                          -16.8734      8.841     -1.909      0.056     -34.214       0.468
gender_M                           19.0058      8.680      2.190      0.029       1.980      36.032
location_code_Rural              4.442e-14   2.04e-14      2.181      0.029    4.47e-15    8.44e-14
location_code_Suburban            152.8495     16.087      9.501      0.000     121.295     184.404
location_code_Urban              -150.7170     20.322     -7.417      0.000    -190.578    -110.856
marital_status_Divorced           -20.9338     13.458     -1.555      0.120     -47.332       5.465
marital_status_Married            -44.9323      9.695     -4.635      0.000     -63.948     -25.916
marital_status_Single              67.9986     10.850      6.267      0.000      46.717      89.280
policy_type_Corporate Auto         -5.2386      9.940     -0.527      0.598     -24.735      14.258
policy_type_Personal Auto          -2.3458      8.652     -0.271      0.786     -19.316      14.625
policy_type_Special Auto            9.7169     14.639      0.664      0.507     -18.997      38.431
policy_Corporate L1                 0.3358     20.037      0.017      0.987     -38.967      39.638
policy_Corporate L2                -4.3023     17.627     -0.244      0.807     -38